In [15]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import pandas as pd
import numpy as np
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from sqlalchemy import create_engine
from selenium.common.exceptions import NoSuchElementException
from sqlalchemy.types import Integer
import nltk
from nltk.corpus import names
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
from timeit import default_timer as timer


In [25]:
start = timer()#contador de tiempo para ver cuanto tarda el codigo en ejecutarse
Municipalidades=['MU003','MU011','MU178','MU313','MU329','MU181','MU324']# aca se declara el codigo de las municipalidades a trabajar
#MU003 = Municipalidad de alto biobio
#MU011 = Municipalidad de Arauco
#MU178 = Municipalidad de Nacimiento
#MU313 = Municipalidad de Talcahuano
#MU329 = Municipalidad de Tucapel
#MU181 = Municipalidad de Negrete
#MU324 = Municipalidad de Tome

path=["//*[@id='A2248:form:j_idt30:0:datalist:3:j_idt33:0:ta-link']","//*[@id='A2248:form:j_idt30:0:datalist:3:j_idt33:1:ta-link']"]#se guarda el xpath para el personal contrata y planta
encontrado=2#se declara variable para ver si hay mas paginas dentro de la tabla o no
engine = create_engine('sqlite:///Comunas.db', echo=True)#se crea la base de datos
sqlite_connection = engine.connect()#se conecta
pagina=1#contador
enc=1#contador
#se crea la inteligencia artificial para establecer los sexos
labeled_names = ([(name, 'H') for name in names.words('male.txt')] + [(name, 'M') for name in names.words('female.txt')]) 
random.shuffle(labeled_names)
featuresets = [({'last_five_letters':n[-5:]}, gender) for (n, gender) in labeled_names]   
train_set, test_set = featuresets[100:], featuresets[:100] 
classifier = nltk.NaiveBayesClassifier.train(train_set) 
#se empiezan a crear los ciclos que permitiran avanzar en las paginas de la municipalidad
for h in range(7):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    #driver = webdriver.Chrome('./chromedriver.exe')
    Direccion="https://www.portaltransparencia.cl/PortalPdT/pdtta?codOrganismo=%s"  % Municipalidades[h]
    driver.get(Direccion)
    sleep(2)
    
    for o in range(2):
        #se declaran las variables iniciales, se hace dentro del for para que se limpien cada vez que se ejecute el ciclo
        año=[]
        mes=[]
        est=[]
        nombre=[]
        cargo=[]
        grado=[]
        profesion=[]
        region=[]
        Asignaciones=[]
        unidad=[]
        rembru=[]
        remliq=[]
        extrad=[]
        extran=[]
        extraf=[]
        fechaini=[]
        fechater=[]
        declaraciones=[]
        viaticos=[]
        observaciones=[]
        Clasificacion=[]
        #se avanza hacia las tablas de las municipalidades mediante la utilizacion de xpath
        element = driver.find_element_by_xpath(path[o])
        driver.execute_script("arguments[0].click();", element)
        element = driver.find_element_by_xpath("//*[@id='A2248:form-visualizar:j_idt57_list']/li[1]/a")
        driver.execute_script("arguments[0].click();", element)
        element = driver.find_element_by_xpath("//*[@id='A2248:form-visualizar:j_idt57_list']/li[1]/a")
        driver.execute_script("arguments[0].click();", element)
        element =driver.find_element_by_xpath("//*[@id='A2248:form-visualizar:j_idt57_list']/li[1]/a")
        driver.execute_script("arguments[0].click();", element)
        #se declara un try, exeption para ver si tiene mas paginas o no la tabla, si esque tiene rescata todas las paginas, si no solo 1
        try:
            element =driver.find_element_by_xpath("//*[@id='A2248:form-visualizar:datosplantilla_paginator_top']/span[4]/span")
            encontrado=1
        except NoSuchElementException:
            encontrado=0

        if encontrado == 1: 
            while enc == 1: 

                try:
                    nextpage="//*[@id='A2248:form-visualizar:datosplantilla_paginator_top']/span[3]/span[%s]" %(pagina)
                    element =driver.find_element_by_xpath(nextpage)
                    element.click() 

                    sleep(5)
                    a=1
                    b=1
                    Fila = driver.find_elements_by_xpath("//*[@id='A2248:form-visualizar:datosplantilla_data']/tr")
                    Columna= driver.find_elements_by_xpath("//*[@id='A2248:form-visualizar:datosplantilla_data']/tr[1]/td")
                    for Fil in  Fila:
                        for Col in Columna:

                            url= "//*[@id='A2248:form-visualizar:datosplantilla_data']/tr[%s]/td[%s]" %(a,b)
                            Lista2= driver.find_element_by_xpath(url).text
                            #se rescatan el valor de las 20 Columnas por fila
                            if b==1:
                                año.append(Lista2)
                            if b==2:
                                mes.append(Lista2)
                            if b==3:
                                est.append(Lista2)
                            if b==4:
                                nombre.append(Lista2)
                                minus = Lista2.lower()
                                Clas=(classifier.classify({'last_five_letters':minus[-5:]}))
                                Clasificacion.append(Clas)
                            if b==5:
                                cargo.append(Lista2)
                            if b==6:
                                grado.append(Lista2)
                            if b==7:
                                profesion.append(Lista2)
                            if b==8:
                                region.append(Lista2)
                            if b==9:
                                Asignaciones.append(Lista2)
                            if b==10:
                                unidad.append(Lista2)
                            if b==11:
                                rembru.append(Lista2)
                            if b==12:
                                remliq.append(Lista2)
                            if b==13:
                                extrad.append(Lista2)
                            if b==14:
                                extran.append(Lista2)
                            if b==15:
                                extraf.append(Lista2)
                            if b==16:
                                fechaini.append(Lista2)
                            if b==17:
                                fechater.append(Lista2)
                            if b==18:
                                declaraciones.append(Lista2)
                            if b==19:
                                viaticos.append(Lista2)
                            if b==20:
                                observaciones.append(Lista2)

                            b=b+1
                        a=a+1
                        b=1
                    pagina=pagina+1
                    a=1
                    b=1
                except NoSuchElementException:
                        print ("no hay mas paginas encontradas")
                        enc=0

        if encontrado == 0: 
                    a=1
                    b=1
                    Fila = driver.find_elements_by_xpath("//*[@id='A2248:form-visualizar:datosplantilla_data']/tr")
                    Columna= driver.find_elements_by_xpath("//*[@id='A2248:form-visualizar:datosplantilla_data']/tr[1]/td")
                    for Fil in  Fila:
                        for Col in Columna:

                            url= "//*[@id='A2248:form-visualizar:datosplantilla_data']/tr[%s]/td[%s]" %(a,b)
                            Lista2= driver.find_element_by_xpath(url).text
                            if b==1:
                                año.append(Lista2)
                            if b==2:
                                mes.append(Lista2)
                            if b==3:
                                est.append(Lista2)
                            if b==4:
                                nombre.append(Lista2)
                                minus = Lista2.lower()
                                Clas=(classifier.classify({'last_five_letters':minus[-5:]}))
                                Clasificacion.append(Clas)
                                
                            if b==5:
                                cargo.append(Lista2)
                            if b==6:
                                grado.append(Lista2)
                            if b==7:
                                profesion.append(Lista2)
                            if b==8:
                                region.append(Lista2)
                            if b==9:
                                Asignaciones.append(Lista2)
                            if b==10:
                                unidad.append(Lista2)
                            if b==11:
                                rembru.append(Lista2)
                            if b==12:
                                remliq.append(Lista2)
                            if b==13:
                                extrad.append(Lista2)
                            if b==14:
                                extran.append(Lista2)
                            if b==15:
                                extraf.append(Lista2)
                            if b==16:
                                fechaini.append(Lista2)
                            if b==17:
                                fechater.append(Lista2)
                            if b==18:
                                declaraciones.append(Lista2)
                            if b==19:
                                viaticos.append(Lista2)
                            if b==20:
                                observaciones.append(Lista2)
                                

                            b=b+1
                        a=a+1
                        b=1
        #se crea el diccionario de datos
        Nombres={'Año': año,'Mes': mes,'Estamento':est, 'Nombre Completo':nombre, 'Cargo o funcion':cargo, 'Grado':grado,
        'Calificacion profesional':profesion, 'Region':region, 'Asignaciones especiales': Asignaciones, 'Unidad monetaria':unidad,
        'Remuneración bruta mensualizada':rembru,'Remuneración líquida mensualizada':remliq, 'Montos y horas extraordinarias diurnas':extrad,
        'Montos y horas extraordinarias nocturnas':extran,'Montos y horas extraordinarias festivas':extraf, 'Fecha de inicio dd/mm/aa': fechaini,
        'Fecha de término dd/mm/aa':fechater,'Declaración de intereses y patrimonio (publicación por buena práctica) ':declaraciones,
        'Viáticos':viaticos,'Observaciones':observaciones,'Clasificacion Sexo': Clasificacion}
        driver.get(Direccion)
        dt=pd.DataFrame(Nombres)
        dt.index = np.arange(1, len(dt)+1)
        #se crean if para ver si los datos pertenecen a planta o contrato y se crean el nombre para guardar el excel y la base de datos
        if o==0:
            guardarexcel="Regiones/ContrataComunaCod= %s.xlsx" % Municipalidades[h]
            guardarbase="ContrataComunaCod=%s.db" % Municipalidades[h]
        if o==1:
            guardarexcel="Regiones/PlantaComunaCod= %s.xlsx" % Municipalidades[h]
            guardarbase="PlantaComunaCod%s.db" % Municipalidades[h]
        #se pasa el df al excel
        dt.to_excel(guardarexcel)
        #se pasa el df a la base de datos
        sqlite_table = guardarbase
        dt.to_sql(sqlite_table, sqlite_connection, if_exists='replace')
    driver.close();   
        
end = timer()#finalizacion de tiempo de ejecucion
print(end - start)
Tiempo= end - start
# se crea el archivo log para ver el tiempo de ejecucion, la fecha y si estuvo correcto todo
with open('RegistroEjecucion.log', 'a') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Proceso Exitoso, Tiempo de ejecucion de: ' ,Tiempo , datetime.now()])
sqlite_connection.close()


2020-07-16 22:22:48,501 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-16 22:22:48,502 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:22:48,503 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-16 22:22:48,503 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:23:17,395 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU003.db")
2020-07-16 22:23:17,396 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:23:17,397 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU003.db")
2020-07-16 22:23:17,398 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:23:17,399 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:23:17,400 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:23:17,401 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU003.db")
2020-07-

2020-07-16 22:23:17,476 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-16 22:23:17,484 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:23:17,485 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:23:33,894 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU003.db")
2020-07-16 22:23:33,895 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:23:33,896 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU003.db")
2020-07-16 22:23:33,896 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:23:33,898 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:23:33,899 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:23:33,900 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU003.db")
2020-07-16 22:23:33,901 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:23:33,906 INFO sqlalchemy.engine.base.Engine SELECT 

2020-07-16 22:23:33,975 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-16 22:23:33,983 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:23:33,984 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:24:53,397 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU011.db")
2020-07-16 22:24:53,398 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:24:53,398 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU011.db")
2020-07-16 22:24:53,399 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:24:53,401 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:24:53,401 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:24:53,404 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU011.db")
2020-07-16 22:24:53,405 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:24:53,407 INFO sqlalchemy.engine.base.Engin

2020-07-16 22:24:53,480 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-16 22:24:53,487 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:24:53,488 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:25:34,316 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU011.db")
2020-07-16 22:25:34,317 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:25:34,318 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU011.db")
2020-07-16 22:25:34,319 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:25:34,320 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:25:34,321 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:25:34,324 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU011.db")
2020-07-16 22:25:34,328 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:25:34,333 INFO sqlalchemy.engine.base.Engine SELECT 

2020-07-16 22:25:34,447 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-16 22:25:34,456 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:25:34,460 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:26:12,295 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU178.db")
2020-07-16 22:26:12,296 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:26:12,297 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU178.db")
2020-07-16 22:26:12,298 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:26:12,299 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:26:12,300 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:26:12,302 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU178.db")
2020-07-16 22:26:12,304 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:26:12,309 INFO sqlalchemy.engine.base.Engin

...

2020-07-16 22:33:07,449 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-16 22:33:07,456 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:33:07,457 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:33:42,167 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU329.db")
2020-07-16 22:33:42,168 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:33:42,169 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU329.db")
2020-07-16 22:33:42,170 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:33:42,171 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:33:42,172 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:33:42,174 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU329.db")
2020-07-16 22:33:42,177 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:33:42,181 INFO sqlalchemy.engine.base.Engine SELECT 

2020-07-16 22:33:42,258 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-16 22:33:42,267 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:33:42,268 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:05,739 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU181.db")
2020-07-16 22:34:05,740 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:05,742 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU181.db")
2020-07-16 22:34:05,744 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:05,746 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:34:05,749 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:05,752 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU181.db")
2020-07-16 22:34:05,753 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:05,758 INFO sqlalchemy.engine.base.Engin

2020-07-16 22:34:05,859 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:25,869 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU181.db")
2020-07-16 22:34:25,869 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:25,870 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU181.db")
2020-07-16 22:34:25,871 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:25,873 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:34:25,874 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:25,876 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU181.db")
2020-07-16 22:34:25,877 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:34:25,881 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'PlantaComunaCodMU181.db' AND type = 'table'
2020-07-16 22:34:25,883 INFO sqlalchemy.engine

2020-07-16 22:35:31,799 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:31,800 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU324.db")
2020-07-16 22:35:31,801 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:31,802 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:35:31,804 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:31,806 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ContrataComunaCod=MU324.db")
2020-07-16 22:35:31,808 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:31,810 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'ContrataComunaCod=MU324.db' AND type = 'table'
2020-07-16 22:35:31,813 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:31,816 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("ContrataComunaCod=MU324.db")
2020-07-16 22:35:31,817 INFO

2020-07-16 22:35:31,911 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-16 22:35:31,921 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:35:31,922 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:39,444 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU324.db")
2020-07-16 22:35:39,445 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:39,446 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU324.db")
2020-07-16 22:35:39,447 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:39,448 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-07-16 22:35:39,449 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:39,450 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PlantaComunaCodMU324.db")
2020-07-16 22:35:39,451 INFO sqlalchemy.engine.base.Engine ()
2020-07-16 22:35:39,453 INFO sqlalchemy.engine.base.Engine SELECT 